# The Rank/Atop operator: `⍤`

> An algorithm must be seen to be believed.
\--_Don Knuth_

The somewhat startled-looking _Rank_ operator, `⍤`, has a reputation for being one of the harder ones to grasp. It sits on the 'J' key, a handy mnemonic, as it was an invention borrowed from the [J-language](https://www.jsoftware.com/#/). Well, it makes sense to _me_.

If `⍤` is given a right operand _function_ it becomes _Atop_. We'll talk about that after we've covered the _Rank_ version.

Other resources on _Rank_ and _Atop_:
* [APL Wiki](https://aplwiki.com/wiki/Rank_(operator))
* Dyalog docs [Rank](https://help.dyalog.com/latest/Content/Language/Primitive%20Operators/Rank.htm), [Atop](https://help.dyalog.com/latest/index.htm#Language/Primitive%20Operators/Atop.htm)
* Webinars: [Rank Operator](https://dyalog.tv/Webinar/?v=IBct81IopRk), [Advanced Rank](https://dyalog.tv/Webinar/?v=5wW76XX0kqk), [Rank and Dyadic Transpose](https://dyalog.tv/Webinar/?v=zBqdeDJPPRc)
* APL Orchard cultivations: [Basic Rank](https://chat.stackexchange.com/rooms/52405/conversation/lesson-32-basic-use-of-), [Advanced Rank](https://chat.stackexchange.com/rooms/52405/conversation/lesson-33-advanced-use-of-), [Atop and Over](https://chat.stackexchange.com/rooms/52405/conversation/lesson-47-atop--and-over-)
* Roger Hui: [Rank Operator: An Idea Worth <s>Stealing</s> Borrowing](https://www.jsoftware.com/papers/rank/)

In [1]:
⎕IO ← 0
]box on
]rows on

Rebuilding user command cache... 
done

Was ON

Was OFF

In essence, the simple use of _Rank_ drills into the argument array and applies its operand to sub-cells of a specified rank only. 

Consider the following rank 3 array:

In [2]:
]DISPLAY m ← 3 3 3⍴27?27
≢⍴m

┌┌→───────┐
↓↓ 3 0 21│
││19 15 9│
││24 6 17│
││ │
││26 8 20│
││ 4 14 12│
││11 23 5│
││ │
││18 7 2│
││13 16 1│
││22 25 10│
└└~───────┘

3

Let's say we want to drop the first row of each layer of this array. In this case, we could use a bracket-axis specification to drop:

In [3]:
1↓[1]m ⍝ Apply drop ↓ to axis 1, i.e. the y-axis, with ⎕IO←0

19 15 9
24 6 17

 4 14 12
11 23 5

13 16 1
22 25 10

However, this is now `⎕IO` dependent, and the bracket-axis spec has some other undesirable properties, including the fact that it cannot be applied to user-defined functions. 

The _Rank_ operator allows us to solve this in a different way:

In [4]:
1↓⍤2⊢m

19 15 9
24 6 17

 4 14 12
11 23 5

13 16 1
22 25 10

You can think of _Rank_ in this case being closer to the problem statement: drop the first major cell of each _sub-cell_ that has a rank one less than the argument. Or: in our 3D array, apply the drop to each of its constituent 2D arrays.

Note that _Rank_ doesn't equal _depth_, and this is part of the reason why the _Rank_ operator can be confusing. So given a complex vector that contains a variety of scalars and higher-rank arrays, you can't use the _Rank_ operator to selectively apply a function to, say, just the vectors.

In [5]:
⎕ ← V ← (1 2 3)(2 2⍴1 1 1 1)(1)

┌─────┬───┬─┐
│1 2 3│1 1│1│
│ │1 1│ │
└─────┴───┴─┘

In [6]:
≢⍤1⊢V ⍝ NOTE: this is wrong!

3

In this case, `V` is of course already a rank 1 array (i.e. a vector). But wait, there is more confusion to be had! The right operand doesn't even have to be a scalar at all.

We already saw that if the right operand of `⍤` is a scalar, we are specifying which rank sub cells we want a function to apply to. For dyadic usage, we instead specify which sub-cells of the left argument should be paired up with which sub-cells of the right argument. It takes a while to wrap your head around this.

Here's an example:

In [7]:
1 2 3,⍤0 1 ⊢ 'hello'

1 hello
2 hello
3 hello

This says that the _left_ argument of the derived function should be paired at rank 0 (that is each element) with the _right_ argument at rank 1 (that is the whole vector in this case). If this feels reminiscent of an outer product, you're not wrong.

In other words, _Rank pairs up_ the arguments of the function it derives. We can examine which elements would be paired up by using the following incantation in place of our function:

In [8]:
rankpairs ← ⊂⍤,⍥⊂

if you know what that means, you probably don't need to read this tutorial. If not, don't worry -- treat it as a debug statement:

In [9]:

1 2 3 rankpairs⍤0 1 ⊢ 'hello'

┌─────────┬─────────┬─────────┐
│┌─┬─────┐│┌─┬─────┐│┌─┬─────┐│
││1│hello│││2│hello│││3│hello││
│└─┴─────┘│└─┴─────┘│└─┴─────┘│
└─────────┴─────────┴─────────┘

In the section on [array indexing](./indexing.ipynb) earlier, we introduced the concept of _Sane Indexing_ without any further explanation on how it was working. Here it is again, as a [tacit](./tacit.ipynb) function:

In [10]:
I←⌷⍨∘⊃⍨⍤0 99 ⍝ Sane indexing, a.k.a select

Now that we understand a bit about _Rank_, we can at least make a start in taking that apart. So we have a _Rank_ 0 99 to the right, stating that we should combine the left argument at rank 0 (scalars) with the right argument at "rank 99". Rank 99 means "full rank" (Dyalog has an _actual_ max rank of 15, so we could have used any number greater than or equal to 15). So we should combine "each scalar" to the left with "the whole thing, at whatever rank it happens to be" to the right.

The actual indexing is done by the leading _Squad_ (`⌷`). Let's de-selfie the expression and make it a dfn. Although we haven't covered [tacit](./tacit.ipynb) functions yet, we can have the interpreter help us out a bit to see how that expression actually parses:

In [11]:
]box on -trains=tree ⍝ Visualise tacit functions as a parse tree

Was ON -trains=tree

In [12]:
⌷⍨∘⊃⍨⍤0 99

⍤
 ┌┴┐
 ⍨ 0 99
 ┌─┘
 ∘
 ┌┴┐
 ⍨ ⊃
┌─┘
⌷

If we separate out the _Rank_ expression, after a bit of head-scratching, we arrive at

In [13]:
sane←{(⊃⍺)⌷⍵}

In [14]:
3 sane 'abcdefg'

d

In [15]:
1 2 3 sane⍤0 99⊢'abcdefg'

bcd

Hopefully, we can now see a bit clearer how it works -- we simply pass the disclosed left argument vector to _Squad_, and _Rank_ does the appropriate combination of left and right: each left element, to the whole (full-rank) of the right.

Here's another example. Given two integer arrays of the same shape, can we replicate (or compress) the rows of one from the other? We might be excused if we think that we can replicate-reduce, but this doesn't work:

In [16]:
]DISPLAY A ← 3 4⍴2 1 1 1 1 2 1 1 1 1 2 1
]DISPLAY B ← 3 4⍴1 2 3 4 1 2 3 4 1 2 3 4

┌→──────┐
↓2 1 1 1│
│1 2 1 1│
│1 1 2 1│
└~──────┘

┌→──────┐
↓1 2 3 4│
│1 2 3 4│
│1 2 3 4│
└~──────┘

(note: all expressions in the next cell will generate syntax or rank errors)

In [17]:
A∘//B     ⍝ Nope... SYNTAX ERROR
{A/⍵}/B   ⍝ Nope... RANK ERROR
A∘{⍺/⍵}/B ⍝ Nope... SYNTAX ERROR

SYNTAX ERROR: The function does not take a left argument
      A∘//B     ⍝ Nope... SYNTAX ERROR
        ∧


RANK ERROR
      {A/⍵}/B   ⍝ Nope... RANK ERROR
        ∧


SYNTAX ERROR: The function does not take a left argument
      A∘{⍺/⍵}/B ⍝ Nope... SYNTAX ERROR
            ∧


Instead, we need to reach for _Rank_:

In [18]:
A/⍤1⊢B

1 1 2 3 4
1 2 2 3 4
1 2 3 3 4

If we spell it out in plain words, perhaps the use of _Rank_ suggests itself: combine each major cell on the left with the corresponding major cell on the right, and apply the dyadic function "replicate". Every time you think "combine each major cell on the left...", you should think _Rank_.

As we've seen elsewhere, APL has a somewhat unholy mix of leading and trailing axis operators, mainly for historical reasons. _Rank_ allows us to stick to the leading axis versions [[Kromberg and Hui](https://dl.acm.org/doi/pdf/10.1145/3386319)]:

|Leading axis|Trailling axis|Operation|
|:-----------|:-------------|:--------|
|(⊖⍤1)⍵      |⌽⍵           |reverse  |
|⍺(⊖⍤1)⍵     |⍺⌽⍵           |rotate (scalar ⍺)|
|(f⌿⍤1)⍵      |f/⍵           | reduce  |
|(f⍀⍤1)⍵      |f\⍵           | scan |


## Relative rank

The right operand can also contain negative numbers, which are taken to be rank relative to the argument, rather than an absolute specification:

In [19]:
+/⍤¯1⊢3 3⍴⍳9              ⍝ Sum at rank one less than the rank of the matrix, i.e. along each row
]display 2 3 4⍴⍳24
]display +/⍤¯2⊢2 3 4⍴⍳24  ⍝ Sum at rank two less than the rank of the cube, i.e. along rows of each layer

3 12 21

┌┌→──────────┐
↓↓ 0 1 2 3│
││ 4 5 6 7│
││ 8 9 10 11│
││ │
││12 13 14 15│
││16 17 18 19│
││20 21 22 23│
└└~──────────┘

┌→───────┐
↓ 6 22 38│
│54 70 86│
└~───────┘

## Rank drills 

In his paper [APL Excercises](https://www.jsoftware.com/papers/APL_exercises/), Roger Hui sets out the following drills for _Rank_. See if you can predict the results before revealing the answers:

In [20]:
,t←2 3 4⍴⍳24

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23

In [21]:
,⍤3⊢t
]DISPLAY ,⍤2⊢t
]DISPLAY ,⍤1⊢t

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23

┌→──────────────────────────────────┐
↓ 0 1 2 3 4 5 6 7 8 9 10 11│
│12 13 14 15 16 17 18 19 20 21 22 23│
└~──────────────────────────────────┘

┌┌→──────────┐
↓↓ 0 1 2 3│
││ 4 5 6 7│
││ 8 9 10 11│
││ │
││12 13 14 15│
││16 17 18 19│
││20 21 22 23│
└└~──────────┘

In [22]:
⊂⍤3⊢t
⊂⍤2⊢t
⊂⍤1⊢t

┌───────────┐
│ 0 1 2 3│
│ 4 5 6 7│
│ 8 9 10 11│
│ │
│12 13 14 15│
│16 17 18 19│
│20 21 22 23│
└───────────┘

┌─────────┬───────────┐
│0 1 2 3│12 13 14 15│
│4 5 6 7│16 17 18 19│
│8 9 10 11│20 21 22 23│
└─────────┴───────────┘

┌───────────┬───────────┬───────────┐
│0 1 2 3 │4 5 6 7 │8 9 10 11 │
├───────────┼───────────┼───────────┤
│12 13 14 15│16 17 18 19│20 21 22 23│
└───────────┴───────────┴───────────┘

In [23]:
↑ ,¨ ⊂⍤3⊢t
]DISPLAY ↑ ,¨ ⊂⍤2⊢t
]DISPLAY ↑ ,¨ ⊂⍤1⊢t

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23

┌→──────────────────────────────────┐
↓ 0 1 2 3 4 5 6 7 8 9 10 11│
│12 13 14 15 16 17 18 19 20 21 22 23│
└~──────────────────────────────────┘

┌┌→──────────┐
↓↓ 0 1 2 3│
││ 4 5 6 7│
││ 8 9 10 11│
││ │
││12 13 14 15│
││16 17 18 19│
││20 21 22 23│
└└~──────────┘

In [24]:
]DISPLAY +⌿⍤3⊢t
]DISPLAY +⌿⍤2⊢t
]DISPLAY +⌿⍤1⊢t

┌→──────────┐
↓12 14 16 18│
│20 22 24 26│
│28 30 32 34│
└~──────────┘

┌→──────────┐
↓12 15 18 21│
│48 51 54 57│
└~──────────┘

┌→───────┐
↓ 6 22 38│
│54 70 86│
└~───────┘

In [25]:
]DISPLAY ⊖⍤3⊢t
]DISPLAY ⊖⍤2⊢t
]DISPLAY ⊖⍤1⊢t

┌┌→──────────┐
↓↓12 13 14 15│
││16 17 18 19│
││20 21 22 23│
││ │
││ 0 1 2 3│
││ 4 5 6 7│
││ 8 9 10 11│
└└~──────────┘

┌┌→──────────┐
↓↓ 8 9 10 11│
││ 4 5 6 7│
││ 0 1 2 3│
││ │
││20 21 22 23│
││16 17 18 19│
││12 13 14 15│
└└~──────────┘

┌┌→──────────┐
↓↓ 3 2 1 0│
││ 7 6 5 4│
││11 10 9 8│
││ │
││15 14 13 12│
││19 18 17 16│
││23 22 21 20│
└└~──────────┘

If you got all those right, well done -- Roger's paper has got many more for you to pit your wits against.

## Atop

As we mentioned above, from version 18 of Dyalog, `⍤` can also be _Atop_ if given a right operand function instead of array. We'll see more of different kinds of _Atop_ later in the [tacit](./tacit.ipynb) chapter, but for now it's a functional composition rule, stating that (for the dyadic case),
```apl
X f⍤g Y → f X g Y
```

It is similar enough to _Jot_, `∘`, to be confusing. Whilst monadic `∘` is an atop, dyadic `∘` is a _beside_. Behold:
```apl
f∘g X → f(gX) → fgX
f⍤g X → f(gX) → fgX
X f∘g Y → X f g Y
X f⍤g Y → f X g Y
```

Another way to think about it is that in the dyadic case, the difference between `∘` and `⍤` is which operand function (`f` or `g`) gets the left argument. In the monadic case, _Atop_ is equivalent for `∘` and `⍤`, so it's worth using `⍤` for both monadic and dyadic atops for consistency. 

Here is a simple example of a dyadic _Atop_,

In [26]:
1 2 3 -⍤+ 4 5 6 ⍝ Negate the sum of two vectors

¯5 ¯7 ¯9

which, following the `X f⍤g Y → f X g Y` rule, is the same as

In [27]:
-1 2 3+4 5 6 ⍝ Negate the sum of two vectors

¯5 ¯7 ¯9

or, as a tacit formulation (which we'll cover in more detail later),

In [28]:
1 2 3 (-+) 4 5 6 ⍝ Negate the sum of two vectors

¯5 ¯7 ¯9